In [1]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore
from rdflib import Graph, Namespace, URIRef

from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems
from Marc_to_Bibframe.Work.work import Work
from Marc_to_Bibframe.Instance.instance import Instance

import pysolr
from Solr.solr import create_doc


In [87]:
#JENA
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/bibframe/update')
query_endpoint = 'http://localhost:3030/bibframe/query'
update_endpoint = 'http://localhost:3030/bibframe/update'
store.open((query_endpoint, update_endpoint))



In [2]:
#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":3,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-3"}},\n  "status":"OK"}\n'

In [3]:
def Make_Graph(nt, bf, count ):
    G1 = "PREFIX bk: <http://bibliokeia.com/bibframe/"+bf+"""/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {
        GRAPH bk:"""

    G2 = " {"

    G3 = """}
    }"""

    G = G1+str(count)+" { \n"+nt+"}}"

    return G

In [15]:
def Upadate(path_marc, count, shelf):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(records.index(record))
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)

        #JENA
        BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
        BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/{count}")
        #Work
        work = Work(count, workMarc, BFwork, BFinstance)
        work.serialize(f'out/E1/P3/works/{count}.ttl', format='turtle')
        nt = work.serialize(format='nt')
        W = Make_Graph(nt, "work", count)
        #store.update(W)
        #Instance
        instance = Instance(count, workMarc, instanceMarc, itemsMarc, BFwork, BFinstance, shelf)
        i = instance.get('instance')
        #i.serialize(f'out/E1/P2/instances/{count}.ttl', format='turtle')
        i_nt = i.serialize(format='nt')
        I = Make_Graph(i_nt, "instance", count)
        #store.update(I)
        #Items
        items = instance.get('items')
        for item, register in items:
            item.serialize(f'out/E1/P2/items/{register}.ttl', format='turtle')
            item_nt = item.serialize(format='nt')
            Item = Make_Graph(item_nt, "item", register)
            #store.update(Item)

            
        count += 1

path_marc = r'Input/E1/P3/koha.xml'
count = 43
Upadate(path_marc, count, "E1.P3")

0
Pesquisas amazônicas


AssertionError: Subject None must be an rdflib term

In [10]:
docs[0]

{'id': 19,
 'call': '001.4 M916',
 'shelf': 'E1.P2',
 'title': 'Resumos',
 'responsibilities': 'I Mostra Técnico-Científica da Universidade do Amazonas, Manaus, AM, 1999.',
 'place': 'Manaus',
 'publisher': 'UA',
 'year': '1999',
 'subject': ['Ciência--Amazonas--Congressos',
  'Tecnologia--Amazonas--Congressos'],
 'type': 'Livro',
 'items': ['05-0436'],
 'author': None}

In [10]:
solr.add(docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":379}}\n'

In [9]:
docs[9]

{'id': 28,
 'call': '001.4098115 F144f',
 'shelf': 'E1.P2',
 'title': 'FADESP, Fundação de Amaparo e Desenvolvimento da Pesquisa, Belém-Pará.',
 'responsibilities': False,
 'place': 'Belém',
 'publisher': 'A Fundação',
 'year': '1984',
 'subject': ['Pesquisa--Pará'],
 'type': 'Livro',
 'items': ['84-0568'],
 'author': None}

In [13]:
path_marc = r'Koha/E1/P2/koha.xml'
marc_file = parse(path_marc)
records = marc_file.getElementsByTagName('record')

count = 3
record = records[count]
marcxml = etree.fromstring(record.toxml())
workMarc = MarcWork(marcxml)
instanceMarc = MarcInstance(marcxml)
itemsMarc = MarcItems(marcxml)
BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/{count}")
work = Work(count, workMarc, BFwork, BFinstance)
instance = Instance(count, workMarc, instanceMarc, itemsMarc, BFwork, BFinstance, 'E1.P2')

In [14]:
year = instanceMarc.Publication().get('year')
year

'1996'

In [16]:
marcxml.find("datafield/[@tag='260']"\
                                "/subfield/[@code='c']").text

'[1984?].'

In [22]:
year.replace('[', '').replace('?]', '')

'1984'